# V 1.1

### A versão 1.1 tem o formato completo do cromagrama e usa a minha detecção de acordes

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment

In [22]:
def cria_chroma_from_sample(sample, sample_rate):

    y_harmonic = librosa.effects.harmonic(sample)

    # Compute chroma features from the harmonic signal
    # Transforma cada faixa de frequencia em notas musicais
    #chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
    #                                        sr=sample_rate)
     
    chromagram = librosa.amplitude_to_db(np.abs(librosa.cqt(y_harmonic, sr=sample_rate)), ref=np.max)
    
    return chromagram

In [17]:
def identifica_fundamental(chroma):

    #critério de seleção: linha com o maior valor
    linhas_somadas = np.sum(chroma, axis=1)

    # argsort() retorna os índices que ordenariam um array.
    # [-2:] pega os dois últimos índices, que correspondem às maiores somas
    i_maiores_somas = np.argsort(linhas_somadas)[-2:]

    # pega uma nota só
    #indice_nota = np.argmax(linhas_somadas) 

    if (i_maiores_somas[0] + 7 ) %12 == i_maiores_somas[1]:
        indice_nota = i_maiores_somas[0]

    elif (i_maiores_somas[1] + 7 ) %12 == i_maiores_somas[0]:
        indice_nota = i_maiores_somas[1]

    else:
        indice_nota = i_maiores_somas[1]


    notas_musicais = {
    0: "Dó",
    1: "Dó#",
    2: "Ré",
    3: "Ré#",
    4: "Mi",
    5: "Fá",
    6: "Fá#",
    7: "Sol",
    8: "Sol#",
    9: "Lá",
    10: "Lá#",
    11: "Si"
    }

    nota_fundamental = notas_musicais[indice_nota%12]
    return nota_fundamental
    

In [18]:
def contar_sequencias_notas(lista):
    # recebe uma sequencia de notas e transforma em uma matriz 2x2 com a nota e a quantidade de tempos que ela se repetiu
    
    contagens_sequenciais = []
    atual = lista[0]
    contagem = 0

    for elemento in lista:
        if elemento == atual:
            contagem += 1
        else:
            contagens_sequenciais.append([atual, contagem])
            atual = elemento
            contagem = 1
    contagens_sequenciais.append([atual, contagem])  # Para o último elemento

    # Sua lista de entrada
    #entrada = [1, 1, 1, 2, 2, 3, 3, 3, 3, 1, 1, 1, 1, 2, 2]
    #saida   = [[1, 3], [2, 2], [3, 4], [1, 4], [2, 2]]

    # Chamar a função para contar sequências
    #resultado = contar_sequencias_notas(entrada)

    return contagens_sequenciais

In [19]:
def final_process(file = str):
    # Carregar o arquivo mp3
    audio = AudioSegment.from_file(file, format="mp3")

    # Definindo a duração de cada segmento em milissegundos
    #duracao_segmento = 167  # 1/6 segundos
    duracao_segmento = 500  # 1/6 segundos

    notas_fundamentais_acordes = []

    # Segmentando o áudio
    for i in range(0, len(audio), duracao_segmento):
        segmento = audio[i:i + duracao_segmento]
        print(i, i+duracao_segmento)

        sample = np.array(segmento.get_array_of_samples()).astype(np.float32) / 32768
        sample_rate = segmento.frame_rate

        chromagram = cria_chroma_from_sample(sample=sample, sample_rate=sample_rate)

        fundamental = identifica_fundamental(chroma=chromagram)

        notas_fundamentais_acordes.append(fundamental)

    
    notas_fundamentais_acordes = contar_sequencias_notas(notas_fundamentais_acordes)
    return(notas_fundamentais_acordes)

In [23]:
notas_final = final_process('teste2_1min.mp3')

0 500
500 1000
1000 1500
1500 2000
2000 2500
2500 3000
3000 3500
3500 4000
4000 4500
4500 5000
5000 5500
5500 6000
6000 6500
6500 7000
7000 7500
7500 8000
8000 8500
8500 9000
9000 9500
9500 10000
10000 10500
10500 11000
11000 11500
11500 12000
12000 12500
12500 13000
13000 13500
13500 14000
14000 14500
14500 15000
15000 15500
15500 16000
16000 16500
16500 17000
17000 17500
17500 18000
18000 18500
18500 19000
19000 19500
19500 20000
20000 20500
20500 21000
21000 21500
21500 22000
22000 22500
22500 23000
23000 23500
23500 24000
24000 24500
24500 25000
25000 25500
25500 26000
26000 26500
26500 27000
27000 27500
27500 28000
28000 28500
28500 29000
29000 29500
29500 30000
30000 30500
30500 31000
31000 31500
31500 32000
32000 32500
32500 33000
33000 33500
33500 34000
34000 34500
34500 35000
35000 35500
35500 36000
36000 36500
36500 37000
37000 37500
37500 38000
38000 38500
38500 39000
39000 39500
39500 40000
40000 40500
40500 41000
41000 41500
41500 42000
42000 42500
42500 43000
43000 43500


as notas estão indo entre 7 e 7 unidades de tempo escolhidas. Com variação de + - 1

Bug das quintas foi quase completamente corrigido

In [24]:
notas_final

[['Sol', 1],
 ['Fá#', 1],
 ['Sol', 3],
 ['Lá', 2],
 ['Dó', 2],
 ['Lá', 2],
 ['Fá', 5],
 ['Dó', 5],
 ['Dó#', 1],
 ['Sol', 5],
 ['Lá', 1],
 ['Dó', 2],
 ['Lá', 1],
 ['Dó', 1],
 ['Fá', 5],
 ['Dó', 6],
 ['Sol', 5],
 ['Lá', 5],
 ['Fá#', 1],
 ['Fá', 5],
 ['Dó', 2],
 ['Sol', 1],
 ['Mi', 1],
 ['Sol', 6],
 ['Sol#', 1],
 ['Lá', 2],
 ['Mi', 1],
 ['Lá', 1],
 ['Sol', 1],
 ['Fá', 5],
 ['Mi', 1],
 ['Dó', 2],
 ['Sol', 1],
 ['Dó', 1],
 ['Sol', 6],
 ['Lá', 2],
 ['Dó', 1],
 ['Lá', 1],
 ['Sol#', 1],
 ['Fá', 3],
 ['Dó', 1],
 ['Fá', 1],
 ['Mi', 1],
 ['Dó', 5],
 ['Sol', 5],
 ['Lá', 3],
 ['Dó', 1],
 ['Lá', 1],
 ['Fá#', 1],
 ['Fá', 2]]